<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">


# Introduction to APIs
*Authors: dorkydragon and GA*

![](imgs/data_is_around.png)

----

## The `requests` Library
The `requests` library is a library for submitting HTTP requests from Python. Despite its frequent use, it's not included in the Python standard library. You'll need to `pip install requests` yourself.
![](imgs/pokeapi.png)

API documentation can be found at https://pokeapi.co/

In [30]:
import pandas as pd
import numpy as np
import requests
import time

In [1]:
# Create url for API call.
base_url = 'https://pokeapi.co/api/v2/'

In [4]:
# Submit request
res = requests.get(base_url + 'pokemon/ditto')

In [5]:
# Request response code
res.status_code

200

In [6]:
# Text of request
res.text

'{"abilities":[{"ability":{"name":"limber","url":"https://pokeapi.co/api/v2/ability/7/"},"is_hidden":false,"slot":1},{"ability":{"name":"imposter","url":"https://pokeapi.co/api/v2/ability/150/"},"is_hidden":true,"slot":3}],"base_experience":101,"forms":[{"name":"ditto","url":"https://pokeapi.co/api/v2/pokemon-form/132/"}],"game_indices":[{"game_index":76,"version":{"name":"red","url":"https://pokeapi.co/api/v2/version/1/"}},{"game_index":76,"version":{"name":"blue","url":"https://pokeapi.co/api/v2/version/2/"}},{"game_index":76,"version":{"name":"yellow","url":"https://pokeapi.co/api/v2/version/3/"}},{"game_index":132,"version":{"name":"gold","url":"https://pokeapi.co/api/v2/version/4/"}},{"game_index":132,"version":{"name":"silver","url":"https://pokeapi.co/api/v2/version/5/"}},{"game_index":132,"version":{"name":"crystal","url":"https://pokeapi.co/api/v2/version/6/"}},{"game_index":132,"version":{"name":"ruby","url":"https://pokeapi.co/api/v2/version/7/"}},{"game_index":132,"version"

In [7]:
# Bring in the JSON!
ditto = res.json()
ditto.keys()

dict_keys(['abilities', 'base_experience', 'forms', 'game_indices', 'height', 'held_items', 'id', 'is_default', 'location_area_encounters', 'moves', 'name', 'order', 'past_types', 'species', 'sprites', 'stats', 'types', 'weight'])

In [ ]:
# Since we've converted the JSON -> dict, we know how to work with this!


In [9]:
# Height, Weight
print(ditto['height'])
print(ditto['weight'])

3
40


## Challenge
---
What moves can Pikachu learn? In the cells below, do the following:
1. Use `requests` library to get Pikachu's data
1. Convert the JSON into a Python dictionary.
1. Use list comprehension to get just the names of each move. Your result should look like this:

```python
['mega-punch',
 'pay-day',
 'thunder-punch',
 'slam',
 'mega-kick',
 'headbutt',
 'body-slam',
 'take-down',
 ...
]
```

In [13]:
# Use requests library to get Pikachu's data
res = requests.get(base_url + 'pokemon/pikachu')

In [14]:
# Convert the JSON into a Python dictionary.
pikachu = res.json()

In [16]:
# Use list comprehension to get just the names of each move.
[m['move']['name'] for m in  pikachu['moves']][:10]

['mega-punch',
 'pay-day',
 'thunder-punch',
 'slam',
 'mega-kick',
 'headbutt',
 'body-slam',
 'take-down',
 'double-edge',
 'tail-whip']

## Creating a `pandas` DataFrame from JSON
---
To create a DataFrame, we simply need a list of dictionaries. Let's try creating a DataFrame from Pikachu's abilities.

In [19]:
pika_df = pd.DataFrame(pikachu['abilities'])
pika_df

,ability,is_hidden,slot
0,"{'name': 'static', 'url': 'https://pokeapi.co/...",False,1
1,"{'name': 'lightning-rod', 'url': 'https://poke...",True,3


That's all fine and good. But notice each `ability` is a dictionary. In the cell below, let's extract the `name` from each ability and set it as its own column.

In [21]:
pika_df['name'] = pika_df['ability'].map(lambda ab: ab['name'])

In [22]:
pika_df.head()

,ability,is_hidden,slot,name
0,"{'name': 'static', 'url': 'https://pokeapi.co/...",False,1,static
1,"{'name': 'lightning-rod', 'url': 'https://poke...",True,3,lightning-rod


In [24]:
pika_df['ability'].map(lambda ab: ab['url'])

0     https://pokeapi.co/api/v2/ability/9/
1    https://pokeapi.co/api/v2/ability/31/
Name: ability, dtype: object

## Challenge: get a description for each ability.
---

If you're a Pokemon neophyte (like me), you might not know what the **static** is. Fortunately, we can use the `url` from our `ability` dictionary to make an API request. Because we're hitting several URLs in a loop, we want to throttle our requests so as not to overwhelm the API.

In the cell below, create a function that retrieves the `effect`. NOTE: There are multiple languages, choose one.

In [34]:
def get_effect_from_ability(url):
    print(url)
    time.sleep(2)
    res = requests.get(url)
    ability = res.json()
    descriptions = [e['effect'] for e in ability['effect_entries'] if e['language']['name'] == 'en']
    desc = descriptions[0] if len(descriptions) > 0 else np.nan
    return desc
get_effect_from_ability(url = 'https://pokeapi.co/api/v2/ability/31/')

https://pokeapi.co/api/v2/ability/31/


"All other Pokémon's single-target electric-type moves are redirected to this Pokémon if it is an eligible target.  Other Pokémon's Electric moves raise this Pokémon's Special Attack one stage, negating any other effect on it, and cannot miss it.\n\nIf the move's intended target also has this ability, the move is not redirected.  When multiple Pokémon with this ability are possible targets for redirection, the move is redirected to the one with the highest Speed stat, or, in the case of a tie, to a random tied Pokémon.  follow me takes precedence over this ability.\n\nIf the Pokémon is a ground-type and thus immune to Electric moves, its immunity prevents the Special Attack boost."

Now use the `.map()` method to get the effect for each ability.

In [38]:
pika_df['effect'] = pika_df['ability'].map(lambda ab: ab['url']).map(get_effect_from_ability)
pika_df

https://pokeapi.co/api/v2/ability/9/
https://pokeapi.co/api/v2/ability/31/


,ability,is_hidden,slot,name,effect
0,"{'name': 'static', 'url': 'https://pokeapi.co/...",False,1,static,Whenever a move makes contact with this Pokémo...
1,"{'name': 'lightning-rod', 'url': 'https://poke...",True,3,lightning-rod,All other Pokémon's single-target electric-typ...



## On to Reddit!

----

<img src="imgs/reddit_logo.png" style="width: 250px; margin: 0 0 20px 0;" alt="Reddit Logo" />

We'll use the PushShift API. 

[Click for documentation](https://github.com/pushshift/api)

In [42]:
base_url = 'https://api.pushshift.io/reddit/'

## Parameters
---

If we want the post recent posts from /r/Science, we have to use the `subreddit` parameter. The URL would be as follows:

https://api.pushshift.io/reddit/search/submission/?subreddit=science

Parameters are everything after the `?` in a URL. They're sort of like a dictionary, in that each parameter is a key/value pair. (ie `subreddit=science`). 

Multiple parameters are separated with a `&`.

https://api.pushshift.io/reddit/search/submission/?subreddit=science&foo=bar

## Challenge
---
Use the API documentation to answer the following question:

How would we search for the word "election" in [/r/news](https://reddit.com/r/news)? Your answer should be a URL with two parameters.

In [44]:
# NOTE: You already have base_url
base_url + 'search/submission/?subreddit=news&q=election'

'https://api.pushshift.io/reddit/search/submission/?subreddit=news&q=election'

With the `requests` library, we can use the full URL (including params):

```python
requests.get('https://api.pushshift.io/reddit/search/submission/?subreddit=news&q=election')
```

Or (even better), the `requests` will accept the parameters as a dictionary:

```python
params = {
    'subreddit': 'news',
    'q': 'election'
}
requests.get(base_url + 'search/submission/', params=params)
```

In [48]:
# Get the most recent *50* posts from */r/news* containing the word *"election"*

params = {
    'subreddit': 'news',
    'q': 'election',
    'size': 50
}
res = requests.get(base_url + 'search/submission/', params=params)
data = res.json()


In [52]:
# Convert the posts into a DataFrame
df = pd.DataFrame(data['data'])

## Challenge
---
The posts are ordered by `created_utc`. In the cell below, prove that this is true.

In [68]:
(df['created_utc'].sort_values(ascending=False).reset_index(drop=True) == df['created_utc']).sum()

50

In [69]:
last = df['created_utc'].values[-1]
last

1626969901

Use the last date from the previous step to get the **next** 50 posts that occur **before** your variable. Save it as another DataFrame.

In [71]:
params = {
    'subreddit': 'news',
    'q': 'election',
    'size': 50,
    'before': last
}
res = requests.get(base_url + 'search/submission/', params=params)
data = res.json()

Concatenate the two DataFrames together.

In [73]:
pd.concat([
    df, pd.DataFrame(data['data'])
], ignore_index=True)['created_utc'].nunique()

100